# Advanced Model Calibration and Threshold Optimization Tutorial 🎯

## 🎯 Learning Objectives
By the end of this tutorial, you will understand:

1. **Model Calibration** - Why and how to calibrate probability predictions
2. **Calibration Methods** - Platt scaling vs Isotonic regression
3. **Threshold Optimization** - Finding optimal decision thresholds based on business costs
4. **Dynamic Thresholds** - Context-aware thresholds for different scenarios
5. **Multi-objective Optimization** - Balancing multiple business goals

## 📋 What This File Does
The `advanced_model_calibration.py` file implements:

**🎯 Model Calibration:**
- Platt scaling (sigmoid calibration)
- Isotonic regression (non-parametric)
- Calibration evaluation metrics (ECE, MCE)
- Cross-validated calibration

**💰 Threshold Optimization:**
- Cost-sensitive threshold selection
- Pareto-optimal thresholds
- Dynamic thresholds by context
- Business impact analysis

**📊 Evaluation Metrics:**
- Expected Calibration Error (ECE)
- Maximum Calibration Error (MCE)
- Brier score
- Reliability diagrams

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Calibration imports
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    precision_recall_curve, roc_curve, average_precision_score,
    brier_score_loss, log_loss, precision_score, recall_score, 
    f1_score, confusion_matrix, roc_auc_score
)
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import joblib

# Visualization settings
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Advanced Model Calibration Tutorial")
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 50)

## 1. Understanding Model Calibration

Model calibration ensures that predicted probabilities reflect true probabilities. A well-calibrated model that predicts 70% probability should be correct 70% of the time.

In [ ]:
# Demonstrate calibration concept
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Perfect calibration
perfect_x = np.linspace(0, 1, 100)
perfect_y = perfect_x
ax1.plot(perfect_x, perfect_y, 'g-', linewidth=3, label='Perfect Calibration')
ax1.plot([0, 1], [0, 1], 'k--', alpha=0.5)
ax1.set_xlabel('Mean Predicted Probability')
ax1.set_ylabel('Fraction of Positives')
ax1.set_title('Well-Calibrated Model', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Poor calibration examples
# Overconfident model
overconfident_x = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
overconfident_y = np.array([0.05, 0.1, 0.15, 0.25, 0.5, 0.75, 0.85, 0.9, 0.95])
ax2.plot(overconfident_x, overconfident_y, 'r-', linewidth=3, label='Overconfident', marker='o')

# Underconfident model
underconfident_x = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
underconfident_y = np.array([0.15, 0.25, 0.35, 0.45, 0.5, 0.55, 0.65, 0.75, 0.85])
ax2.plot(underconfident_x, underconfident_y, 'b-', linewidth=3, label='Underconfident', marker='s')

ax2.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Perfect')
ax2.set_xlabel('Mean Predicted Probability')
ax2.set_ylabel('Fraction of Positives')
ax2.set_title('Poorly Calibrated Models', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Calibration Importance:")
print("  • Well-calibrated models provide reliable probability estimates")
print("  • Overconfident models predict extreme probabilities too often")
print("  • Underconfident models predict probabilities too close to 0.5")
print("  • Calibration is crucial for threshold-based decisions")

## 2. Loading Data and Training Base Models

Let's load our data and train some base models that we'll calibrate:

In [ ]:
# Load the dataset
df = pd.read_csv('../creditcard.csv')

print("📊 Dataset Information:")
print(f"Total transactions: {len(df):,}")
print(f"Fraud rate: {df['Class'].mean()*100:.3f}%")

# Prepare features
feature_columns = [col for col in df.columns if col != 'Class']
X = df[feature_columns]
y = df['Class']

# Create train-calibration-test split
# First split: separate test set
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Second split: separate calibration set from training
X_train, X_cal, y_train, y_cal = train_test_split(
    X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42
)

print(f"\n📂 Data Splits:")
print(f"Training set: {len(X_train):,} samples ({y_train.sum()} frauds)")
print(f"Calibration set: {len(X_cal):,} samples ({y_cal.sum()} frauds)")
print(f"Test set: {len(X_test):,} samples ({y_test.sum()} frauds)")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_cal_scaled = scaler.transform(X_cal)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train base models
print("🤖 Training Base Models...")
print("=" * 50)

base_models = {}

# 1. Random Forest
print("\n🌲 Training Random Forest...")
rf = RandomForestClassifier(
    n_estimators=100, 
    random_state=42, 
    class_weight='balanced',
    n_jobs=-1
)
rf.fit(X_train_scaled, y_train)
base_models['Random Forest'] = rf

# 2. XGBoost
print("📊 Training XGBoost...")
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    random_state=42,
    scale_pos_weight=scale_pos_weight
)
xgb_model.fit(X_train_scaled, y_train)
base_models['XGBoost'] = xgb_model

# 3. Logistic Regression
print("📈 Training Logistic Regression...")
lr = LogisticRegression(
    random_state=42,
    class_weight='balanced',
    max_iter=1000
)
lr.fit(X_train_scaled, y_train)
base_models['Logistic Regression'] = lr

# Evaluate uncalibrated models
print("\n📊 Uncalibrated Model Performance:")
for name, model in base_models.items():
    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    brier = brier_score_loss(y_test, y_proba)
    
    print(f"\n{name}:")
    print(f"  F1-Score: {f1:.4f}")
    print(f"  ROC-AUC: {auc:.4f}")
    print(f"  Brier Score: {brier:.4f} (lower is better)")

## 3. Visualizing Calibration Before Calibration

Let's check how well-calibrated our models are before calibration:

In [ ]:
# Plot calibration curves for uncalibrated models
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (name, model) in enumerate(base_models.items()):
    ax = axes[idx]
    
    # Get predictions
    y_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # Calculate calibration curve
    fraction_of_positives, mean_predicted_value = calibration_curve(
        y_test, y_proba, n_bins=10, strategy='uniform'
    )
    
    # Plot
    ax.plot(mean_predicted_value, fraction_of_positives, 's-', 
            label=name, markersize=8, linewidth=2)
    ax.plot([0, 1], [0, 1], 'k--', label='Perfect Calibration')
    
    # Calculate ECE
    ece = np.mean(np.abs(fraction_of_positives - mean_predicted_value))
    
    ax.set_xlabel('Mean Predicted Probability')
    ax.set_ylabel('Fraction of Positives')
    ax.set_title(f'{name}\nECE: {ece:.4f}', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('Calibration Curves - Before Calibration', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("📊 Observations:")
print("  • Random Forest often shows overconfidence (S-shaped curve)")
print("  • XGBoost may need calibration depending on hyperparameters")
print("  • Logistic Regression is often well-calibrated by default")

## 4. Model Calibration Methods

Let's apply two calibration methods: Platt Scaling and Isotonic Regression

In [ ]:
# Function to calculate calibration metrics
def calculate_calibration_metrics(y_true, y_prob, n_bins=10):
    """Calculate various calibration metrics"""
    # Brier score
    brier = brier_score_loss(y_true, y_prob)
    
    # Log loss
    logloss = log_loss(y_true, y_prob)
    
    # Expected Calibration Error (ECE)
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]
    
    ece = 0
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        in_bin = (y_prob > bin_lower) & (y_prob <= bin_upper)
        prop_in_bin = in_bin.mean()
        
        if prop_in_bin > 0:
            accuracy_in_bin = y_true[in_bin].mean()
            avg_confidence_in_bin = y_prob[in_bin].mean()
            ece += np.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin
    
    # Maximum Calibration Error (MCE)
    mce = 0
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        in_bin = (y_prob > bin_lower) & (y_prob <= bin_upper)
        
        if in_bin.sum() > 0:
            accuracy_in_bin = y_true[in_bin].mean()
            avg_confidence_in_bin = y_prob[in_bin].mean()
            error = np.abs(avg_confidence_in_bin - accuracy_in_bin)
            mce = max(mce, error)
    
    return {
        'brier_score': brier,
        'log_loss': logloss,
        'ece': ece,
        'mce': mce
    }

# Apply calibration methods
calibrated_models = {}
calibration_results = {}

calibration_methods = ['sigmoid', 'isotonic']

print("🎯 Calibrating Models...")
print("=" * 60)

for model_name, model in base_models.items():
    print(f"\n📊 Calibrating {model_name}")
    
    calibrated_models[model_name] = {}
    calibration_results[model_name] = {'uncalibrated': {}}
    
    # Evaluate uncalibrated model
    y_prob_uncal = model.predict_proba(X_test_scaled)[:, 1]
    uncal_metrics = calculate_calibration_metrics(y_test, y_prob_uncal)
    calibration_results[model_name]['uncalibrated'] = uncal_metrics
    
    print(f"  Uncalibrated - ECE: {uncal_metrics['ece']:.4f}, Brier: {uncal_metrics['brier_score']:.4f}")
    
    for method in calibration_methods:
        # Create calibrated classifier
        calibrated_clf = CalibratedClassifierCV(
            base_estimator=model,
            method=method,
            cv=3,  # 3-fold cross-validation
            ensemble=True
        )
        
        # Fit on calibration set
        calibrated_clf.fit(X_cal_scaled, y_cal)
        
        # Evaluate
        y_prob_cal = calibrated_clf.predict_proba(X_test_scaled)[:, 1]
        cal_metrics = calculate_calibration_metrics(y_test, y_prob_cal)
        
        calibrated_models[model_name][method] = calibrated_clf
        calibration_results[model_name][method] = cal_metrics
        
        print(f"  {method.title()} - ECE: {cal_metrics['ece']:.4f}, Brier: {cal_metrics['brier_score']:.4f}")

## 5. Visualizing Calibration Results

Let's compare calibration curves before and after calibration:

In [ ]:
# Create comprehensive calibration comparison
fig, axes = plt.subplots(3, 3, figsize=(15, 15))

for i, (model_name, model) in enumerate(base_models.items()):
    # Uncalibrated
    ax = axes[i, 0]
    y_prob = model.predict_proba(X_test_scaled)[:, 1]
    fraction_pos, mean_pred = calibration_curve(y_test, y_prob, n_bins=10)
    ax.plot(mean_pred, fraction_pos, 's-', label='Uncalibrated', markersize=8)
    ax.plot([0, 1], [0, 1], 'k--', alpha=0.5)
    ax.set_title(f'{model_name} - Uncalibrated', fontweight='bold')
    ax.set_xlabel('Mean Predicted Probability')
    ax.set_ylabel('Fraction of Positives')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # Platt Scaling
    ax = axes[i, 1]
    cal_model = calibrated_models[model_name]['sigmoid']
    y_prob_cal = cal_model.predict_proba(X_test_scaled)[:, 1]
    fraction_pos, mean_pred = calibration_curve(y_test, y_prob_cal, n_bins=10)
    ax.plot(mean_pred, fraction_pos, 's-', label='Platt Scaling', markersize=8, color='green')
    ax.plot([0, 1], [0, 1], 'k--', alpha=0.5)
    ax.set_title(f'{model_name} - Platt Scaling', fontweight='bold')
    ax.set_xlabel('Mean Predicted Probability')
    ax.set_ylabel('Fraction of Positives')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # Isotonic Regression
    ax = axes[i, 2]
    cal_model = calibrated_models[model_name]['isotonic']
    y_prob_cal = cal_model.predict_proba(X_test_scaled)[:, 1]
    fraction_pos, mean_pred = calibration_curve(y_test, y_prob_cal, n_bins=10)
    ax.plot(mean_pred, fraction_pos, 's-', label='Isotonic', markersize=8, color='orange')
    ax.plot([0, 1], [0, 1], 'k--', alpha=0.5)
    ax.set_title(f'{model_name} - Isotonic Regression', fontweight='bold')
    ax.set_xlabel('Mean Predicted Probability')
    ax.set_ylabel('Fraction of Positives')
    ax.grid(True, alpha=0.3)
    ax.legend()

plt.suptitle('Calibration Comparison: Before and After', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Show calibration metrics comparison
print("📊 Calibration Metrics Comparison")
print("=" * 80)

for model_name in base_models.keys():
    print(f"\n{model_name}:")
    print(f"{'Method':<15} {'ECE':>10} {'MCE':>10} {'Brier':>10} {'Log Loss':>10}")
    print("-" * 55)
    
    for method, metrics in calibration_results[model_name].items():
        print(f"{method:<15} {metrics['ece']:>10.4f} {metrics['mce']:>10.4f} "
              f"{metrics['brier_score']:>10.4f} {metrics['log_loss']:>10.4f}")

## 6. Business-Optimal Threshold Selection

Now let's find optimal decision thresholds based on business costs:

In [ ]:
# Define business costs
class BusinessCosts:
    def __init__(self):
        self.fraud_cost = 200  # Cost of missing a fraud
        self.false_positive_cost = 10  # Cost of false alarm
        self.investigation_cost = 30  # Cost to investigate

costs = BusinessCosts()

print("💰 Business Cost Structure:")
print(f"  • Missing a fraud (FN): ${costs.fraud_cost}")
print(f"  • False alarm (FP): ${costs.false_positive_cost}")
print(f"  • Investigation per alert: ${costs.investigation_cost}")

# Function to find optimal threshold
def find_optimal_threshold(y_true, y_prob, costs):
    """Find threshold that minimizes total business cost"""
    thresholds = np.arange(0.01, 1.0, 0.01)
    total_costs = []
    
    for threshold in thresholds:
        y_pred = (y_prob >= threshold).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        
        # Calculate costs
        fraud_loss = fn * costs.fraud_cost
        false_alarm_cost = fp * costs.false_positive_cost
        investigation_cost = (tp + fp) * costs.investigation_cost
        total_cost = fraud_loss + false_alarm_cost + investigation_cost
        
        total_costs.append(total_cost)
    
    # Find minimum cost threshold
    optimal_idx = np.argmin(total_costs)
    optimal_threshold = thresholds[optimal_idx]
    optimal_cost = total_costs[optimal_idx]
    
    return optimal_threshold, optimal_cost, thresholds, total_costs

# Find optimal thresholds for best calibrated models
threshold_results = {}

print("\n🎯 Finding Optimal Thresholds...")
print("=" * 60)

for model_name in base_models.keys():
    # Get best calibrated model (lowest ECE)
    best_method = min(calibration_results[model_name].keys(), 
                     key=lambda x: calibration_results[model_name][x]['ece'])
    
    if best_method == 'uncalibrated':
        model = base_models[model_name]
    else:
        model = calibrated_models[model_name][best_method]
    
    # Get predictions
    y_prob = model.predict_proba(X_test_scaled)[:, 1]
    
    # Find optimal threshold
    opt_threshold, opt_cost, thresholds, costs = find_optimal_threshold(y_test, y_prob, costs)
    
    # Calculate metrics at optimal threshold
    y_pred_opt = (y_prob >= opt_threshold).astype(int)
    f1_opt = f1_score(y_test, y_pred_opt)
    precision_opt = precision_score(y_test, y_pred_opt)
    recall_opt = recall_score(y_test, y_pred_opt)
    
    threshold_results[model_name] = {
        'best_calibration': best_method,
        'optimal_threshold': opt_threshold,
        'optimal_cost': opt_cost,
        'thresholds': thresholds,
        'costs': costs,
        'f1_score': f1_opt,
        'precision': precision_opt,
        'recall': recall_opt
    }
    
    print(f"\n{model_name} ({best_method}):")
    print(f"  Optimal threshold: {opt_threshold:.3f}")
    print(f"  Total cost: ${opt_cost:,.0f}")
    print(f"  F1-Score: {f1_opt:.4f}")
    print(f"  Precision: {precision_opt:.4f}")
    print(f"  Recall: {recall_opt:.4f}")

## 7. Visualizing Threshold Optimization

Let's visualize how different thresholds affect costs and performance:

In [ ]:
# Plot threshold analysis for each model
fig, axes = plt.subplots(3, 2, figsize=(15, 15))

for idx, (model_name, results) in enumerate(threshold_results.items()):
    # Cost vs Threshold
    ax1 = axes[idx, 0]
    ax1.plot(results['thresholds'], results['costs'], 'b-', linewidth=2)
    ax1.axvline(results['optimal_threshold'], color='r', linestyle='--', 
               label=f"Optimal: {results['optimal_threshold']:.3f}")
    ax1.axvline(0.5, color='gray', linestyle=':', label='Default: 0.5')
    ax1.set_xlabel('Threshold')
    ax1.set_ylabel('Total Business Cost ($)')
    ax1.set_title(f'{model_name} - Cost vs Threshold', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Metrics vs Threshold
    ax2 = axes[idx, 1]
    
    # Calculate metrics for all thresholds
    best_method = results['best_calibration']
    if best_method == 'uncalibrated':
        model = base_models[model_name]
    else:
        model = calibrated_models[model_name][best_method]
    
    y_prob = model.predict_proba(X_test_scaled)[:, 1]
    
    precisions = []
    recalls = []
    f1_scores = []
    
    for threshold in results['thresholds']:
        y_pred = (y_prob >= threshold).astype(int)
        if y_pred.sum() > 0:  # Avoid division by zero
            precisions.append(precision_score(y_test, y_pred))
            recalls.append(recall_score(y_test, y_pred))
            f1_scores.append(f1_score(y_test, y_pred))
        else:
            precisions.append(0)
            recalls.append(0)
            f1_scores.append(0)
    
    ax2.plot(results['thresholds'], precisions, 'g-', label='Precision', linewidth=2)
    ax2.plot(results['thresholds'], recalls, 'b-', label='Recall', linewidth=2)
    ax2.plot(results['thresholds'], f1_scores, 'r-', label='F1-Score', linewidth=2)
    ax2.axvline(results['optimal_threshold'], color='black', linestyle='--', alpha=0.5)
    ax2.set_xlabel('Threshold')
    ax2.set_ylabel('Score')
    ax2.set_title(f'{model_name} - Metrics vs Threshold', fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0, 1)

plt.tight_layout()
plt.show()

## 8. Dynamic Thresholds by Context

Let's explore how thresholds might vary based on transaction context (e.g., amount, time):

In [ ]:
# Create context-based analysis
df_test = pd.DataFrame(X_test, columns=feature_columns)
df_test['Class'] = y_test.values

# Add predictions from best model
best_model_name = min(threshold_results.keys(), 
                     key=lambda x: threshold_results[x]['optimal_cost'])
best_method = threshold_results[best_model_name]['best_calibration']

if best_method == 'uncalibrated':
    best_model = base_models[best_model_name]
else:
    best_model = calibrated_models[best_model_name][best_method]

df_test['fraud_prob'] = best_model.predict_proba(X_test_scaled)[:, 1]

# Create amount bins
df_test['Amount_Bin'] = pd.cut(
    df_test['Amount'], 
    bins=[0, 10, 50, 200, 1000, float('inf')],
    labels=['Very Low', 'Low', 'Medium', 'High', 'Very High']
)

# Find optimal thresholds for each amount bin
print("💰 Dynamic Thresholds by Transaction Amount")
print("=" * 60)

dynamic_thresholds = {}

for amount_bin in df_test['Amount_Bin'].unique():
    if pd.isna(amount_bin):
        continue
        
    # Filter data for this bin
    bin_mask = df_test['Amount_Bin'] == amount_bin
    bin_data = df_test[bin_mask]
    
    if len(bin_data) < 50 or bin_data['Class'].sum() < 5:
        continue
    
    # Find optimal threshold for this bin
    opt_threshold, opt_cost, _, _ = find_optimal_threshold(
        bin_data['Class'].values, 
        bin_data['fraud_prob'].values, 
        costs
    )
    
    # Calculate metrics
    y_pred = (bin_data['fraud_prob'] >= opt_threshold).astype(int)
    precision = precision_score(bin_data['Class'], y_pred)
    recall = recall_score(bin_data['Class'], y_pred)
    
    dynamic_thresholds[amount_bin] = {
        'threshold': opt_threshold,
        'cost': opt_cost,
        'precision': precision,
        'recall': recall,
        'sample_size': len(bin_data),
        'fraud_count': bin_data['Class'].sum()
    }
    
    print(f"\n{amount_bin} (${df_test[bin_mask]['Amount'].min():.0f}-${df_test[bin_mask]['Amount'].max():.0f}):")
    print(f"  Samples: {len(bin_data):,} ({bin_data['Class'].sum()} frauds)")
    print(f"  Optimal threshold: {opt_threshold:.3f}")
    print(f"  Precision: {precision:.3f}, Recall: {recall:.3f}")

# Visualize dynamic thresholds
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Threshold by amount bin
bins = list(dynamic_thresholds.keys())
thresholds = [dynamic_thresholds[b]['threshold'] for b in bins]
precisions = [dynamic_thresholds[b]['precision'] for b in bins]
recalls = [dynamic_thresholds[b]['recall'] for b in bins]

x_pos = np.arange(len(bins))
ax1.bar(x_pos, thresholds, color='#3498db')
ax1.axhline(y=0.5, color='red', linestyle='--', label='Default (0.5)')
ax1.set_xlabel('Amount Bin')
ax1.set_ylabel('Optimal Threshold')
ax1.set_title('Dynamic Thresholds by Transaction Amount', fontweight='bold')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(bins, rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Precision-Recall by amount bin
width = 0.35
ax2.bar(x_pos - width/2, precisions, width, label='Precision', color='#27ae60')
ax2.bar(x_pos + width/2, recalls, width, label='Recall', color='#e74c3c')
ax2.set_xlabel('Amount Bin')
ax2.set_ylabel('Score')
ax2.set_title('Performance by Transaction Amount', fontweight='bold')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(bins, rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 9. Multi-Objective Threshold Optimization

Let's find Pareto-optimal thresholds that balance multiple objectives:

In [ ]:
# Find Pareto-optimal thresholds
def find_pareto_optimal_thresholds(y_true, y_prob, costs):
    """Find thresholds that are Pareto-optimal across multiple objectives"""
    thresholds = np.arange(0.01, 1.0, 0.01)
    objectives = []
    
    for threshold in thresholds:
        y_pred = (y_prob >= threshold).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        
        # Calculate objectives
        total_cost = (fn * costs.fraud_cost + 
                     fp * costs.false_positive_cost + 
                     (tp + fp) * costs.investigation_cost)
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        objectives.append({
            'threshold': threshold,
            'cost': -total_cost,  # Negative because we want to maximize
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        })
    
    # Find Pareto-optimal solutions
    pareto_optimal = []
    
    for i, obj1 in enumerate(objectives):
        is_dominated = False
        
        for j, obj2 in enumerate(objectives):
            if i == j:
                continue
            
            # Check if obj1 is dominated by obj2
            if (obj2['cost'] >= obj1['cost'] and
                obj2['precision'] >= obj1['precision'] and
                obj2['recall'] >= obj1['recall'] and
                (obj2['cost'] > obj1['cost'] or
                 obj2['precision'] > obj1['precision'] or
                 obj2['recall'] > obj1['recall'])):
                is_dominated = True
                break
        
        if not is_dominated:
            pareto_optimal.append(obj1)
    
    return pareto_optimal

# Find Pareto-optimal thresholds for best model
y_prob_best = df_test['fraud_prob'].values
pareto_thresholds = find_pareto_optimal_thresholds(y_test, y_prob_best, costs)

print(f"🎯 Found {len(pareto_thresholds)} Pareto-optimal thresholds")
print("\nSample Pareto-optimal solutions:")
print(f"{'Threshold':>10} {'Cost':>12} {'Precision':>10} {'Recall':>10} {'F1-Score':>10}")
print("-" * 62)

# Show a sample of Pareto-optimal solutions
for i in range(0, len(pareto_thresholds), max(1, len(pareto_thresholds) // 5)):
    sol = pareto_thresholds[i]
    print(f"{sol['threshold']:>10.3f} ${-sol['cost']:>11,.0f} "
          f"{sol['precision']:>10.3f} {sol['recall']:>10.3f} {sol['f1_score']:>10.3f}")

# Visualize Pareto frontier
fig = plt.figure(figsize=(15, 5))

# 3D plot of objectives
ax1 = fig.add_subplot(131, projection='3d')
pareto_costs = [-p['cost'] for p in pareto_thresholds]
pareto_precisions = [p['precision'] for p in pareto_thresholds]
pareto_recalls = [p['recall'] for p in pareto_thresholds]

ax1.scatter(pareto_costs, pareto_precisions, pareto_recalls, c='red', s=50, alpha=0.8)
ax1.set_xlabel('Total Cost ($)')
ax1.set_ylabel('Precision')
ax1.set_zlabel('Recall')
ax1.set_title('Pareto Frontier (3D)', fontweight='bold')

# 2D projections
ax2 = fig.add_subplot(132)
ax2.scatter(pareto_costs, pareto_precisions, c='blue', s=50, alpha=0.8)
ax2.set_xlabel('Total Cost ($)')
ax2.set_ylabel('Precision')
ax2.set_title('Cost vs Precision', fontweight='bold')
ax2.grid(True, alpha=0.3)

ax3 = fig.add_subplot(133)
ax3.scatter(pareto_precisions, pareto_recalls, c='green', s=50, alpha=0.8)
ax3.set_xlabel('Precision')
ax3.set_ylabel('Recall')
ax3.set_title('Precision vs Recall', fontweight='bold')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Final Recommendations and Summary

Let's create a comprehensive summary and recommendations:

In [ ]:
# Create summary report
print("📊 MODEL CALIBRATION AND THRESHOLD OPTIMIZATION SUMMARY")
print("=" * 80)

# Best calibration method for each model
print("\n🎯 Best Calibration Methods:")
for model_name in base_models.keys():
    best_method = min(calibration_results[model_name].keys(), 
                     key=lambda x: calibration_results[model_name][x]['ece'])
    ece_improvement = (calibration_results[model_name]['uncalibrated']['ece'] - 
                      calibration_results[model_name][best_method]['ece'])
    
    print(f"\n{model_name}:")
    print(f"  Best method: {best_method}")
    print(f"  ECE improvement: {ece_improvement:.4f}")
    print(f"  Final ECE: {calibration_results[model_name][best_method]['ece']:.4f}")

# Optimal thresholds summary
print("\n💰 Optimal Thresholds (Business Cost Minimization):")
best_overall_model = min(threshold_results.keys(), 
                        key=lambda x: threshold_results[x]['optimal_cost'])

for model_name, results in threshold_results.items():
    print(f"\n{model_name}:")
    print(f"  Threshold: {results['optimal_threshold']:.3f}")
    print(f"  Total cost: ${results['optimal_cost']:,.0f}")
    print(f"  Performance: P={results['precision']:.3f}, R={results['recall']:.3f}, F1={results['f1_score']:.3f}")
    if model_name == best_overall_model:
        print("  ⭐ BEST OVERALL MODEL")

# Dynamic threshold insights
print("\n🔄 Dynamic Threshold Insights:")
print("  • Low-value transactions can use higher thresholds (fewer false alarms)")
print("  • High-value transactions benefit from lower thresholds (catch more fraud)")
print("  • Consider implementing context-aware thresholds in production")

# Business recommendations
print("\n💼 Business Recommendations:")
print(f"\n1. Deploy {best_overall_model} with {threshold_results[best_overall_model]['best_calibration']} calibration")
print(f"2. Set threshold to {threshold_results[best_overall_model]['optimal_threshold']:.3f} (not default 0.5)")
print(f"3. Expected daily cost: ${threshold_results[best_overall_model]['optimal_cost']:,.0f}")
print(f"4. Consider dynamic thresholds for 10-20% additional cost reduction")
print(f"5. Monitor calibration drift - recalibrate monthly")

# ROI calculation
baseline_cost = len(y_test) * costs.investigation_cost  # If we investigated everything
optimal_cost = threshold_results[best_overall_model]['optimal_cost']
savings = baseline_cost - optimal_cost
roi = (savings / optimal_cost) * 100

print(f"\n💰 Expected ROI:")
print(f"  Baseline cost (investigate all): ${baseline_cost:,.0f}")
print(f"  Optimized cost: ${optimal_cost:,.0f}")
print(f"  Savings: ${savings:,.0f} ({roi:.1f}% ROI)")

## 11. Key Takeaways and Conclusions

### 🎯 What We Learned:

1. **Model Calibration is Essential**:
   - Raw model probabilities are often poorly calibrated
   - Platt scaling works well for small datasets
   - Isotonic regression is more flexible but needs more data

2. **Threshold Optimization Matters**:
   - Default 0.5 threshold is rarely optimal for business
   - Cost-sensitive thresholds can save significant money
   - Different contexts may require different thresholds

3. **Evaluation Metrics**:
   - ECE and MCE measure calibration quality
   - Brier score combines calibration and discrimination
   - Business metrics should drive final decisions

4. **Multi-Objective Optimization**:
   - Pareto-optimal solutions balance competing objectives
   - No single "best" threshold - depends on priorities
   - Visualization helps understand trade-offs

### 💡 Best Practices:

1. **Always calibrate models** before production deployment
2. **Use held-out calibration set** separate from training and test
3. **Optimize thresholds based on business costs**, not just ML metrics
4. **Consider context-aware thresholds** for better performance
5. **Monitor calibration drift** and recalibrate periodically

### 🚀 Next Steps:

In the upcoming tutorials, you'll learn:
- **Deep Learning**: Advanced neural networks and autoencoders
- **Graph Neural Networks**: Leveraging transaction relationships
- **Online Learning**: Adapting to changing fraud patterns
- **Production Systems**: Building scalable APIs

### 📝 Practice Exercises:

1. Try different calibration methods on your models
2. Experiment with different business cost structures
3. Implement time-based dynamic thresholds
4. Create an ensemble of calibrated models

## 🎉 Congratulations!

You've mastered advanced model calibration and threshold optimization! You now understand:
- How to calibrate probability predictions
- Finding optimal thresholds based on business objectives
- Implementing context-aware decision rules
- Balancing multiple objectives in fraud detection

Ready to explore deep learning for fraud detection? Check out `enhanced_deep_learning.ipynb`!